In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import os

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

import tensorflow as tf
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from loop import TrainingLoop

In [2]:
data = pd.read_csv('data/spirals.csv')
X = data.drop('label', axis=1).to_numpy()
y = data['label'].to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

In [3]:
tf.random.set_seed(42)
np.random.seed(42)

model = Sequential()
model.add(Dense(7, activation='sigmoid'))
model.add(Dense(7, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))

#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.fit(X_train, y_train, batch_size=16, epochs=100)

In [4]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
loss_function = tf.keras.losses.BinaryCrossentropy()

batch_size = 16
epochs = 150

train_metrics = tf.keras.metrics.CategoricalAccuracy()
val_metrics = tf.keras.metrics.CategoricalAccuracy()

In [5]:
@tf.function
def calc_loss(x_train, y_train):
    with tf.GradientTape() as tape:
        logits = model(x_train, training=False)
        loss_value = loss_function(y_train, logits)
    return loss_value

In [6]:
losses = []
def batch_sorting(data, idx, model, loss_function):
    global losses
    if idx == 0:
        for i in range(len(data)):
            x_batch_train = data[i][0]
            y_batch_train = data[i][1]
            losses.append([i, float(calc_loss(x_batch_train, y_batch_train))])
        losses = sorted(losses, key=lambda x:x[1], reverse=True)
        #print(losses)   

    return_idx = losses[idx][0]

    if idx == len(data)-1:
        losses.clear()
    
    return return_idx

In [7]:
training = TrainingLoop(model, X_train, y_train, loss_function, optimizer, validation_split=0.1, batch_size=batch_size, batch_selection=batch_sorting)
training.train(10)

	  0% | 0/45 [00:00<?, ?it/s]WARNING:tensorflow:Layer dense is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because its dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1/10	Loss: 0.6673: 	100% | 45/45 [00:00<00:00, 85.86it/s]
Epoch 2/10	Loss: 0.6031: 	100% | 45/45 [00:00<00:00, 512.35it/s]
Epoch 3/10	Loss: 0.6236: 	100% | 45/45 [00:00<00:00, 509.44it/s]
Epoch 4/10	Loss: 0.5920: 	100% | 45/45 [00:00<00:00, 531.54it/s]
Epoc

In [8]:
pred = model.predict(X_test)
accuracy_score(y_test, pred.round())

0.875